In [1]:
import os
import zipfile
import json
import pandas as pd
from sklearn.model_selection import train_test_split

## trianing data 전처리
- 데이터의 오류 유형에 관한 질문과 해당 질문에 대한 답변, 답변이 포함된 텍스트 위치를 학습

디렉토리 정보
- input : '../data/original/training/'
- output : '../data/training/'

csv파일 명칭 정보
- training_machine_fact.csv : TL_기계요약문_사실전달형_corrected_type1 ~ TL_기계요약문_사실전달형_corrected_type6
- training_machine_opinion.csv : TL_기계요약문_의견제시형_corrected_type1 ~ TL_기계요약문_의견제시형_corrected_type6
- training_machine_free.csv : TL_기계요약문_자유형_corrected_type1 ~ TL_기계요약문_자유형_corrected_type6
- training_human_fact.csv : TL_사람요약문_사실전달형_corrected_type1 ~ TL_사람요약문_사실전달형_corrected_type6
- training_human_opinion.csv : TL_사람요약문_의견제시형_corrected_type1 ~ TL_사람요약문_의견제시형_corrected_type6
- training_human_free.csv : TL_사람요약문_자유형_corrected_type1 ~ TL_사람요약문_자유형_corrected_type6

In [2]:
base_dir_path = '../data_original/training/'
base_output_dir_path = './data/training/'

In [3]:
def error_type_question(type):
    if type==1:
        return '한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?'
    elif type==2:
        return '단어 선택 오류가 발생한 부분은?'
    elif type==3:
        return '비문이 발생한 부분은?'
    elif type==4:
        return '미완성 또는 불완전한 문장이 발생한 부분은?'
    elif type==5:
        return '키워드 또는 중요 내용 오류가 발생한 부분은?'
    elif type==6:
        return '유사한 내용이 반복되는 오류가 발생한 부분은?'

In [9]:
# TL_사람요약문_사실전달형_corrected_type1 ~ TL_사람요약문_사실전달형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'TL_사람요약문_사실전달형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

            id = data['annotation']['original_summary']['human_abstractive_summary']['author']['id']
            
            original_text = data['original_text']
            corrected_type = type_num
            summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
            question = error_type_question(corrected_type)

            corrected_summary = data['annotation']['corrected_summary']
            corrected_text = corrected_summary[f'corrected_type{type_num}']['errors'][0]['sub']
            error_begin = corrected_summary[f'corrected_type{type_num}']['errors'][0]['begin']
            answers = {'text': [corrected_text], 'answer_start': [error_begin]}
            
            data_list.append({
                'id' : id,
                'original': original_text,
                'summary': summary_text,
                # 'label': corrected_type,
                'question' : question,
                'answers' : answers
            })

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['context'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_human_fact = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_human_fact.shape)
print(data_df_training_human_fact.head())

# # csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_human_fact.csv'

# # 데이터프레임을 CSV 파일로 저장
data_df_training_human_fact.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (3627, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                             answers  \
0          {'text': ['180억원'], 'answer_start': [34]}   
1         {'text': ['발산했다 .'], 'answer_start': [76]}   
2  {'text': ['대상학생들에게  자가운동'], 'answer_start': [87]}   
3            {'text': ['승차후'], 'answer_start': [58]}   
4          {'text': ['배터리사업'], 'answer_start': [71]}   

                                             context  
0  [original]제이테크놀로지(옛 마제스타)가 카지노 자회사 매각을 완료해 재무건...  
1  [original]글로벌 뷰티 브랜드 랑콤이 패션 매거진 <마리끌레르>와 함께한 배...  
2  [original]법동초 시작관내 초 순회 진단. 대덕구가 지난 30일 우송대학교 ...  
3  [original]경기파주시 최종환시장은 지난 4일 신설 마을버스 운행 첫날 시민들...  
4  [original]배터리 셀을 든 최태원(가운데) 회장이 김진영 배터리생산기술본부장...  
데이터가 저장되었습니다.


In [10]:
# TL_사람요약문_의견제시형_corrected_type1 ~ TL_사람요약문_의견제시형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []
# id = 1

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'TL_사람요약문_의견제시형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            id = data['annotation']['original_summary']['human_abstractive_summary']['author']['id']
            original_text = data['original_text']
            corrected_type = type_num
            summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
            question = error_type_question(corrected_type)

            corrected_summary = data['annotation']['corrected_summary']
            corrected_text = corrected_summary[f'corrected_type{type_num}']['errors'][0]['sub']
            error_begin = corrected_summary[f'corrected_type{type_num}']['errors'][0]['begin']
            answers = {'text': [corrected_text], 'answer_start': [error_begin]}
            
            data_list.append({
                'id' : id,
                'original': original_text,
                'summary': summary_text,
                # 'label': corrected_type,
                'question' : question,
                'answers' : answers
            })
            # id = id+1

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['context'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_human_opinion = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_human_opinion.shape)
print(data_df_training_human_opinion.head())

# # csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_human_opinion.csv'

# # 데이터프레임을 CSV 파일로 저장
data_df_training_human_opinion.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (3412, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                        answers  \
0    {'text': ['바로잡는것이'], 'answer_start': [99]}   
1      {'text': ['받지못한'], 'answer_start': [64]}   
2    {'text': ['할것이다.'], 'answer_start': [162]}   
3  {'text': ['대해  적극적인'], 'answer_start': [98]}   
4    {'text': ['응시해야한다'], 'answer_start': [97]}   

                                             context  
0  [original]고용노동부가 최근 국무회의에서 발표한 2011~2020 중장기 인...  
1  [original]지난 3월 20일 서울시의회에서는 촛불혁명으로 어렵사리 진전시켜놓...  
2  [original]일본의 경제보복 사태가 여전히 살얼음판이다. 일본이 금수 초지한 ...  
3  [original]부산시가 해수담수화 시설 산업용수의 수요처로 예상했던 울산 온산미...  
4  [original]북한 비핵화를 위한 하노이 선언이 무산됐다. 전 세계의 뜨거운 관...  
데이터가 저장되었습니다.


In [11]:
# TL_사람요약문_자유형_corrected_type1 ~ TL_사람요약문_자유형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []
# id = 1

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'TL_사람요약문_자유형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            id = data['annotation']['original_summary']['human_abstractive_summary']['author']['id']
            original_text = data['original_text']
            corrected_type = type_num
            summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
            question = error_type_question(corrected_type)

            corrected_summary = data['annotation']['corrected_summary']
            corrected_text = corrected_summary[f'corrected_type{type_num}']['text']
            error_begin = corrected_summary[f'corrected_type{type_num}']['errors'][0]['begin']
            answers = {'text': [corrected_text], 'answer_start': [error_begin]}
            
            data_list.append({
                'id' : id,
                'original': original_text,
                'summary': summary_text,
                # 'label': corrected_type,
                'question' : question,
                'answers' : answers
            })
            # id = id+1

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['context'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_training_human_free = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_training_human_free.shape)
print(data_df_training_human_free.head())

# # csv 파일 경로 설정
csv_path = base_output_dir_path + 'training_human_free.csv'

# # 데이터프레임을 CSV 파일로 저장
data_df_training_human_free.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (1871, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                             answers  \
0  {'text': ['법행위의 피해자가 무직자인 경우에 그 소극적 손해의 산정기준은 ...   
1  {'text': ['개정 법률에 규정된 부칙 규정은 원래 법률의 개정에 따른 당해 ...   
2  {'text': ['상표등록취소심판청구 계속 중 상표권자의 상표권 포기로 상표등록이...   
3  {'text': ['주주명단에는 실주소지 기재에 대한 근거가 주주명단에는 실주소지 ...   
4  {'text': ['취득을 고유업무에 직접 사용하다가 1년 이내에 매각한 경우 구 ...   

                                             context  
0  [original]가. 불법행위의 피해자가 입은 소극적 손해를 산정함에 있어 노동능...  
1  [original]1 지방세법 제109조 제3항 본문은 토지구획정리사업법에 의한 토...  
2  [original]가. 상표법 제43조 제1항 제1호에 의한 상표등록취소심판청구는 ...  
3  [original]가. 제2차 납세의무자에 해당하는 과점주주인 여부는 과세처분을 한...  
4  [original]가. 면제된 취득세, 등록세의 추징에 관한 규정인 구 지방세법(1...  
데이터가 저장되었습니다.


In [12]:
# training 사람요약문 병합

csv_paths = [
    base_output_dir_path + 'training_human_fact.csv',
    base_output_dir_path + 'training_human_opinion.csv',
    base_output_dir_path + 'training_human_free.csv'
]

dataframes = [pd.read_csv(path) for path in csv_paths]
merged_training_human_dataframe = pd.concat(dataframes, ignore_index=True)

# 결과 DataFrame을 새로운 CSV 파일로 저장
output_path = base_output_dir_path + 'training_human_data.csv'
merged_training_human_dataframe.to_csv(output_path, index=False)

print('병합이 완료되었습니다.')
print('shape:', merged_training_human_dataframe.shape)
print(merged_training_human_dataframe.head())


병합이 완료되었습니다.
shape: (8910, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                             answers  \
0          {'text': ['180억원'], 'answer_start': [34]}   
1         {'text': ['발산했다 .'], 'answer_start': [76]}   
2  {'text': ['대상학생들에게  자가운동'], 'answer_start': [87]}   
3            {'text': ['승차후'], 'answer_start': [58]}   
4          {'text': ['배터리사업'], 'answer_start': [71]}   

                                             context  
0  [original]제이테크놀로지(옛 마제스타)가 카지노 자회사 매각을 완료해 재무건...  
1  [original]글로벌 뷰티 브랜드 랑콤이 패션 매거진 <마리끌레르>와 함께한 배...  
2  [original]법동초 시작관내 초 순회 진단. 대덕구가 지난 30일 우송대학교 ...  
3  [original]경기파주시 최종환시장은 지난 4일 신설 마을버스 운행 첫날 시민들...  
4  [original]배터리 셀을 든 최태원(가운데) 회장이 김진영 배터리생산기술본부장...  


## validation data 전처리
- 데이터의 오류 유형에 관한 질문과 해당 질문에 대한 답변, 답변이 포함된 텍스트 위치를 학습

디렉토리 정보
- input : '../data/original/validation/'
- output : '../data/validation/'

csv파일 명칭 정보
- validation_machine_fact.csv : VL_기계요약문_사실전달형_corrected_type1 ~ VL_기계요약문_사실전달형_corrected_type6
- validation_machine_opinion.csv : VL_기계요약문_의견제시형_corrected_type1 ~ VL_기계요약문_의견제시형_corrected_type6
- validation_machine_free.csv : VL_기계요약문_자유형_corrected_type1 ~ VL_기계요약문_자유형_corrected_type6
- validation_human_fact.csv : VL_사람요약문_사실전달형_corrected_type1 ~ VL_사람요약문_사실전달형_corrected_type6
- validation_human_opinion.csv : VL_사람요약문_의견제시형_corrected_type1 ~ VL_사람요약문_의견제시형_corrected_type6
- validation_human_free.csv : VL_사람요약문_자유형_corrected_type1 ~ VL_사람요약문_자유형_corrected_type6

In [14]:
base_dir_path = '../data_original/validation/'
base_output_dir_path = './data/validation/'

In [15]:
# VL_사람요약문_사실전달형_corrected_type1 ~ VL_사람요약문_사실전달형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []
# id = 1

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'VL_사람요약문_사실전달형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            id = data['annotation']['original_summary']['human_abstractive_summary']['author']['id']
            original_text = data['original_text']
            corrected_type = type_num
            summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
            question = error_type_question(corrected_type)

            corrected_summary = data['annotation']['corrected_summary']
            corrected_text = corrected_summary[f'corrected_type{type_num}']['errors'][0]['sub']
            error_begin = corrected_summary[f'corrected_type{type_num}']['errors'][0]['begin']
            answers = {'text': [corrected_text], 'answer_start': [error_begin]}
            
            data_list.append({
                'id' : id,
                'original': original_text,
                'summary': summary_text,
                # 'label': corrected_type,
                'question' : question,
                'answers' : answers
            })
            # id = id+1

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['context'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_human_fact = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_human_fact.shape)
print(data_df_validation_human_fact.head())

# # csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_human_fact.csv'

# # 데이터프레임을 CSV 파일로 저장
data_df_validation_human_fact.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (454, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                            answers  \
0          {'text': ['9시5분'], 'answer_start': [82]}   
1          {'text': ['3배가량'], 'answer_start': [49]}   
2  {'text': ['상승했고 ,증권업계는'], 'answer_start': [115]}   
3    {'text': ['2억3000만원을'], 'answer_start': [105]}   
4        {'text': ['순으로매년'], 'answer_start': [145]}   

                                             context  
0  [original]30일 코스닥 지수는 보합권에서 상승 출발했다. 미국 연방공개시장...  
1  [original]경기도 노후 하수관이 1만㎞에 달하며 지하안전 주의보가 발령된(본...  
2  [original]정부와 여당인 더불어민주당이 증권거래세 폐지를 본격 추진하면서 주...  
3  [original]엄재천 올해 장학생 280명 선발2억3천만원 지급 계획,  22일...  
4  [original]전북경찰이 해마다 업무 중 불의의 사고를 당하는 것으로 나타났다....  
데이터가 저장되었습니다.


In [16]:
# VL_사람요약문_의견제시형_corrected_type1 ~ VL_사람요약문_의견제시형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []
# id = 1

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    human_type_data_path = base_dir_path + f'VL_사람요약문_의견제시형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            id = data['annotation']['original_summary']['human_abstractive_summary']['author']['id']
            original_text = data['original_text']
            corrected_type = type_num
            summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
            question = error_type_question(corrected_type)

            corrected_summary = data['annotation']['corrected_summary']
            corrected_text = corrected_summary[f'corrected_type{type_num}']['errors'][0]['sub']
            error_begin = corrected_summary[f'corrected_type{type_num}']['errors'][0]['begin']
            answers = {'text': [corrected_text], 'answer_start': [error_begin]}
            
            data_list.append({
                'id' : id,
                'original': original_text,
                'summary': summary_text,
                # 'label': corrected_type,
                'question' : question,
                'answers' : answers
            })
            # id = id+1

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['context'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_human_opinion = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_human_opinion.shape)
print(data_df_validation_human_opinion.head())

# # csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_human_opinion.csv'

# # 데이터프레임을 CSV 파일로 저장
data_df_validation_human_opinion.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (426, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                           answers  \
0  {'text': ['광주/전라/제주에서'], 'answer_start': [116]}   
1       {'text': ['도전 이며'], 'answer_start': [116]}   
2        {'text': ['그만쓰는게'], 'answer_start': [80]}   
3      {'text': ['정부,지자체,'], 'answer_start': [61]}   
4         {'text': ['15년전'], 'answer_start': [84]}   

                                             context  
0  [original]01. 지방선거 지지 정당 질문 | 6월 13일 시도지사 등을 새...  
1  [original]최악의 한 해였다. 실적은 급전 직하했고, 주가는 폭락했다. 이제...  
2  [original]직장생활 초기에는 나이를 내세우며 아버지뻘이라고 자처하는 취재원을...  
3  [original]일단 정부는 청년 창업 지원책으로 청년층 취업 문제와 창조경제 실...  
4  [original]백화점과 마트자동차가전 등 각 업계가 총출동한 대한민국 동행세일이...  
데이터가 저장되었습니다.


In [17]:
# VL_사람요약문_자유형_corrected_type1 ~ VL_사람요약문_자유형_corrected_type6

# 데이터프레임을 합칠 때 사용할 리스트
data_frames = []
# id = 1

# 각 타입별 폴더를 반복 처리
for type_num in range(1, 7):
    if type_num == 5 :
        continue
        
    human_type_data_path = base_dir_path + f'VL_사람요약문_자유형_corrected_type{type_num}'
    # 데이터를 저장할 리스트 초기화
    data_list = []

    # 해당 디렉토리의 모든 파일을 가져오고, '.json' 확장자를 가진 파일만 필터링
    json_files = [file for file in os.listdir(human_type_data_path) if file.endswith('.json')]

    # 전체 파일 순회
    for index, filename in enumerate(json_files):
        json_path = os.path.join(human_type_data_path, filename)
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            id = data['annotation']['original_summary']['human_abstractive_summary']['author']['id']
            original_text = data['original_text']
            corrected_type = type_num
            summary_text = data['annotation']['original_summary']['human_abstractive_summary']['text']
            question = error_type_question(corrected_type)

            corrected_summary = data['annotation']['corrected_summary']
            corrected_text = corrected_summary[f'corrected_type{type_num}']['text']
            error_begin = corrected_summary[f'corrected_type{type_num}']['errors'][0]['begin']
            answers = {'text': [corrected_text], 'answer_start': [error_begin]}
            
            data_list.append({
                'id' : id,
                'original': original_text,
                'summary': summary_text,
                # 'label': corrected_type,
                'question' : question,
                'answers' : answers
            })
            # id = id+1

        # 각 파일 처리 후 DataFrame 생성 및 'text' 열 추가
        type_data_df = pd.DataFrame(data_list)
        type_data_df['context'] = '[original]' + type_data_df['original'] + '[/original][summary]' + type_data_df['summary'] + '[/summary]'
        type_data_df.drop(columns=['original', 'summary'], inplace=True)

    # 타입별 데이터프레임을 합친다
    data_frames.append(type_data_df)

# 모든 데이터프레임을 하나로 병합
data_df_validation_human_free = pd.concat(data_frames, ignore_index=True)
print('shape: ', data_df_validation_human_free.shape)
print(data_df_validation_human_free.head())

# # csv 파일 경로 설정
csv_path = base_output_dir_path + 'validation_human_free.csv'

# # 데이터프레임을 CSV 파일로 저장
data_df_validation_human_free.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f'데이터가 저장되었습니다.')

shape:  (234, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                             answers  \
0  {'text': ['갑과 을의 개인적 갈등이 심화되어 회사 내부 관계에까지 영향을 ...   
1  {'text': ['도시재개발법에 따르면 도시의 발전 및 공공복리 증진에 기여한다는...   
2  {'text': ['도시 및 주거환경정비법 제 도시 및 주거환경정비법 제84조는 형...   
3  {'text': ['계약 체결에 관하여 신의성실의 원칙에 기한 선의가 강하게 요청되...   
4  {'text': ['피보험자가 보험계약자에 대하여 그 보증보험계약의 주계약과 관련한...   

                                             context  
0  [original]1 산업재해보상보험법상의 업무상 재해라 함은 업무수행중 그 업무에...  
1  [original]도시재개발법(1995. 12. 29. 법률 제5116호로 전문 개...  
2  [original]1 도시 및 주거환경정비법 제84조는  형법 제129조 내지 제1...  
3  [original]1 생명보험계약은 사람의 생명에 관한 우연한 사고에 대하여 금전을...  
4  [original]1보험계약자의 채무불이행시 계약보증금채권을 담보하는 계약이행보증보...  
데이터가 저장되었습니다.


In [18]:
# validation 사람요약문 병합

csv_paths = [
    base_output_dir_path + 'validation_human_fact.csv',
    base_output_dir_path + 'validation_human_opinion.csv',
    base_output_dir_path + 'validation_human_free.csv'
]

dataframes = [pd.read_csv(path, encoding='utf-8') for path in csv_paths]
merged_validation_human_dataframe = pd.concat(dataframes, ignore_index=True)

# 결과 DataFrame을 새로운 CSV 파일로 저장
output_path = base_output_dir_path + 'training_human_data.csv'
merged_validation_human_dataframe.to_csv(output_path, index=False)

print('병합이 완료되었습니다.')
print('shape:', merged_validation_human_dataframe.shape)
print(merged_validation_human_dataframe.head())


병합이 완료되었습니다.
shape: (1114, 4)
           id                   question  \
0  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
1  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
2  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
3  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   
4  human-2020  한글 맞춤법, 띄어쓰기 오류가 발생한 부분은?   

                                            answers  \
0          {'text': ['9시5분'], 'answer_start': [82]}   
1          {'text': ['3배가량'], 'answer_start': [49]}   
2  {'text': ['상승했고 ,증권업계는'], 'answer_start': [115]}   
3    {'text': ['2억3000만원을'], 'answer_start': [105]}   
4        {'text': ['순으로매년'], 'answer_start': [145]}   

                                             context  
0  [original]30일 코스닥 지수는 보합권에서 상승 출발했다. 미국 연방공개시장...  
1  [original]경기도 노후 하수관이 1만㎞에 달하며 지하안전 주의보가 발령된(본...  
2  [original]정부와 여당인 더불어민주당이 증권거래세 폐지를 본격 추진하면서 주...  
3  [original]엄재천 올해 장학생 280명 선발2억3천만원 지급 계획,  22일...  
4  [original]전북경찰이 해마다 업무 중 불의의 사고를 당하는 것으로 나타났다....  


In [19]:
# CSV 파일 읽기
data = pd.read_csv('./data/training/training_human_data.csv')

# 데이터프레임을 레이블별로 분리
groups = data.groupby('question')

# 각 레이블에서 10%를 테스트 데이터로 추출
test_data_list = []
train_data_list = []

for label, group in groups:
    train, test = train_test_split(group, test_size=0.1, random_state=42, stratify=group['question'])
    test_data_list.append(test)
    train_data_list.append(train)

# 테스트 데이터와 훈련 데이터 합치기
test_data = pd.concat(test_data_list)
train_data = pd.concat(train_data_list)

# CSV 파일로 저장
train_data.to_csv('./data/training/training_human_data_2.csv', index=False)
test_data.to_csv('./data/test/test_human_data.csv', index=False)
